# Exploring Ebay Car Sales Data

I'll work with a dataset of used cars from eBay Kleinanzeigen, a classifieds section of the German eBay website.
The dataset was originally scraped and uploaded to Kaggle by user orgesleka.
The original dataset isn't available on Kaggle anymore, but you can find it [here](https://data.world/data-society/used-cars-data).

In [33]:
import pandas as pd
import numpy as np


autos = pd.read_csv('autos.csv', encoding='Latin-1')
autos.info()
autos.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
dateCrawled            50000 non-null object
name                   50000 non-null object
seller                 50000 non-null object
offerType              50000 non-null object
price                  50000 non-null object
abtest                 50000 non-null object
vehicleType            44905 non-null object
yearOfRegistration     50000 non-null int64
gearbox                47320 non-null object
powerPS                50000 non-null int64
model                  47242 non-null object
odometer               50000 non-null object
monthOfRegistration    50000 non-null int64
fuelType               45518 non-null object
brand                  50000 non-null object
notRepairedDamage      40171 non-null object
dateCreated            50000 non-null object
nrOfPictures           50000 non-null int64
postalCode             50000 non-null int64
lastSeen               50000 non-null obj

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


Our dataset contains 20 columns, most of which are stored as strings. There are a few columns with null values, but no columns have more than ~20% null values. There are some columns that contain dates stored as strings.
We'll start by cleaning the column names to make the data easier to work with.

In [34]:
autos.columns

Index(['dateCrawled', 'name', 'seller', 'offerType', 'price', 'abtest',
       'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS', 'model',
       'odometer', 'monthOfRegistration', 'fuelType', 'brand',
       'notRepairedDamage', 'dateCreated', 'nrOfPictures', 'postalCode',
       'lastSeen'],
      dtype='object')

In [35]:
autos.columns = ['date_crawled', 'name', 'seller', 'offer_type', 'price', 'ab_test', 'vehicle_type', 'registration_year', 'gearbox', 'power_ps', 'model', 'odometer', 'registration_month', 'fuel_type', 'brand', 'unrepaired_damage', 'ad_created', 'num_photos', 'postal_code', 'last_seen']
autos.columns

Index(['date_crawled', 'name', 'seller', 'offer_type', 'price', 'ab_test',
       'vehicle_type', 'registration_year', 'gearbox', 'power_ps', 'model',
       'odometer', 'registration_month', 'fuel_type', 'brand',
       'unrepaired_damage', 'ad_created', 'num_photos', 'postal_code',
       'last_seen'],
      dtype='object')

In [3]:
autos.head()

,date_crawled,name,seller,offer_type,price,ab_test,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,num_photos,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


In [36]:
autos.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
date_crawled,50000,48213,2016-03-21 20:37:19,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
name,50000,38754,Ford_Fiesta,78,NaN,NaN,NaN,NaN,NaN,NaN,NaN
seller,50000,2,privat,49999,NaN,NaN,NaN,NaN,NaN,NaN,NaN
offer_type,50000,2,Angebot,49999,NaN,NaN,NaN,NaN,NaN,NaN,NaN
price,50000,2357,$0,1421,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ab_test,50000,2,test,25756,NaN,NaN,NaN,NaN,NaN,NaN,NaN
vehicle_type,44905,8,limousine,12859,NaN,NaN,NaN,NaN,NaN,NaN,NaN
registration_year,50000,NaN,NaN,NaN,2005.07,105.713,1000,1999,2003,2008,9999
gearbox,47320,2,manuell,36993,NaN,NaN,NaN,NaN,NaN,NaN,NaN
power_ps,50000,NaN,NaN,NaN,116.356,209.217,0,70,105,150,17700


Observations:
seller and offer_type have mostly one value
num_photos needs more investigation
price and odometer columns are numeric values stored as text.

In [8]:
autos['num_photos'].value_counts()

0    50000
Name: num_photos, dtype: int64

In [7]:
autos['seller'].value_counts()

privat        49999
gewerblich        1
Name: seller, dtype: int64

From the investigation, the num_photos column has 0 for every column. We'll drop this column, plus the other two we noted as mostly one value.

In [9]:
autos = autos.drop(['num_photos', 'seller', 'offer_type'], axis=1)

In [10]:
autos.head()

,date_crawled,name,price,ab_test,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,39218,2016-04-01 14:38:50


In [97]:
autos.dtypes

date_crawled          object
name                  object
price                  int64
ab_test               object
vehicle_type          object
registration_year      int64
gearbox               object
power_ps               int64
model                 object
odometer              object
registration_month     int64
fuel_type             object
brand                 object
unrepaired_damage     object
ad_created            object
postal_code            int64
last_seen             object
dtype: object

There are two columns, price and odometer, which are numeric values with extra characters being stored as text. We'll clean and convert these.

In [37]:
autos['price'] = autos['price'].str.replace('$', '')
autos['price'] = autos['price'].str.replace(',', '').astype(int)


In [12]:
autos['price'].head()

0    5000
1    8500
2    8990
3    4350
4    1350
Name: price, dtype: int64

In [38]:
autos['odometer'] = autos['odometer'].str.replace('km', '').str.replace(',','').astype(int)

In [8]:
autos['odometer'].head()

0    150000
1    150000
2     70000
3     70000
4    150000
Name: odometer, dtype: int64

In [39]:
autos.rename({'price_USD':'price'}, axis = 1, inplace = True)
autos.rename({'odometer':'odometer_km'}, axis = 1, inplace = True)


In [10]:
autos.columns

Index(['date_crawled', 'name', 'seller', 'offer_type', 'price', 'ab_test',
       'vehicle_type', 'registration_year', 'gearbox', 'power_ps', 'model',
       'odometer_km', 'registration_month', 'fuel_type', 'brand',
       'unrepaired_damage', 'ad_created', 'num_photos', 'postal_code',
       'last_seen'],
      dtype='object')

In [30]:
autos["odometer_km"].value_counts()

150000    32424
125000     5170
100000     2169
90000      1757
80000      1436
70000      1230
60000      1164
50000      1027
5000        967
40000       819
30000       789
20000       784
10000       264
Name: odometer_km, dtype: int64

We can see that the values in this field are rounded, which might indicate that sellers had to choose from pre-set options for this field. Additionally, there are more high mileage than low mileage vehicles.

In [46]:
print(autos['price'].unique().shape)
print(autos['price'].describe())
autos['price'].value_counts().head(20)

(2357,)
count    5.000000e+04
mean     9.840044e+03
std      4.811044e+05
min      0.000000e+00
25%      1.100000e+03
50%      2.950000e+03
75%      7.200000e+03
max      1.000000e+08
Name: price, dtype: float64


0       1421
500      781
1500     734
2500     643
1000     639
1200     639
600      531
800      498
3500     498
2000     460
999      434
750      433
900      420
650      419
850      410
700      395
4500     394
300      384
2200     382
950      379
Name: price, dtype: int64

There are 1421 cars with 0 price.

In [55]:
autos['price'].value_counts().sort_index(ascending = True).head(10)

0     1421
1      156
2        3
3        1
5        2
8        1
9        1
10       7
11       2
12       3
Name: price, dtype: int64

There are a number of listings with prices below \$30, especially about 1,500 at \$0. There are also a small number of listings with very high values, including 14 at around or over $1 million.
Given that eBay is an auction site, there could legitimately be items where the opening bid is \$1. We will keep the \$1 items, but remove anything above \$350,000, since it seems that prices increase steadily to that number and then jump up to less realistic numbers.

In [40]:
autos = autos[autos['price'].between(1,350000)]
autos['price'].describe()

count     48565.000000
mean       5888.935591
std        9059.854754
min           1.000000
25%        1200.000000
50%        3000.000000
75%        7490.000000
max      350000.000000
Name: price, dtype: float64

There are 5 columns that should represent date values. Some of these columns were created by the crawler, some came from the website itself. We can differentiate by referring to the data dictionary
- `date_crawled`: added by the crawler
- `last_seen`: added by the crawler
- `ad_created`: from the website
- `registration_month`: from the website
- `registration_year`: from the website

Right now, the date_crawled, last_seen, and ad_created columns are all identified as string values by pandas. Because these three columns are represented as strings, we need to convert the data into a numerical representation so we can understand it quantitatively. The other two columns are represented as numeric values, so we can use methods like Series.describe() to understand the distribution without any extra data processing.

In [61]:
autos[['date_crawled', 'ad_created', 'last_seen']][0:5]

,date_crawled,ad_created,last_seen
0,2016-03-26 17:47:46,2016-03-26 00:00:00,2016-04-06 06:45:54
1,2016-04-04 13:38:56,2016-04-04 00:00:00,2016-04-06 14:45:08
2,2016-03-26 18:57:24,2016-03-26 00:00:00,2016-04-06 20:15:37
3,2016-03-12 16:58:10,2016-03-12 00:00:00,2016-03-15 03:16:28
4,2016-04-01 14:38:50,2016-04-01 00:00:00,2016-04-01 14:38:50


In [63]:
autos[['date_crawled', 'ad_created', 'last_seen', 'registration_month', 'registration_year']].dtypes

date_crawled          object
ad_created            object
last_seen             object
registration_month     int64
registration_year      int64
dtype: object

In [65]:
autos['date_crawled'].value_counts().head(20)

2016-03-09 11:54:38    3
2016-03-23 18:39:34    3
2016-03-21 20:37:19    3
2016-03-12 16:06:22    3
2016-04-02 15:49:30    3
2016-03-16 21:50:53    3
2016-03-25 19:57:10    3
2016-03-08 10:40:35    3
2016-03-22 09:51:06    3
2016-04-02 11:37:04    3
2016-03-14 20:50:02    3
2016-03-11 22:38:16    3
2016-03-05 16:57:05    3
2016-03-23 19:38:20    3
2016-03-29 23:42:13    3
2016-03-30 19:48:02    3
2016-04-04 16:40:33    3
2016-03-19 17:36:18    3
2016-04-04 19:49:16    2
2016-03-25 19:53:36    2
Name: date_crawled, dtype: int64

In [12]:
autos['date_crawled'].str[:10].value_counts(normalize=True, dropna=False).sort_index()

2016-03-05    0.025327
2016-03-06    0.014043
2016-03-07    0.036014
2016-03-08    0.033296
2016-03-09    0.033090
2016-03-10    0.032184
2016-03-11    0.032575
2016-03-12    0.036920
2016-03-13    0.015670
2016-03-14    0.036549
2016-03-15    0.034284
2016-03-16    0.029610
2016-03-17    0.031628
2016-03-18    0.012911
2016-03-19    0.034778
2016-03-20    0.037887
2016-03-21    0.037373
2016-03-22    0.032987
2016-03-23    0.032225
2016-03-24    0.029342
2016-03-25    0.031607
2016-03-26    0.032204
2016-03-27    0.031092
2016-03-28    0.034860
2016-03-29    0.034099
2016-03-30    0.033687
2016-03-31    0.031834
2016-04-01    0.033687
2016-04-02    0.035478
2016-04-03    0.038608
2016-04-04    0.036487
2016-04-05    0.013096
2016-04-06    0.003171
2016-04-07    0.001400
Name: date_crawled, dtype: float64

The site was crawled daily over a month period from March to April 2016. The distribution of listings crawled on each day remains the same.

In [29]:
autos['ad_created'].str[:10].value_counts(normalize=True, dropna=False).sort_index()

2015-06-11    0.00002
2015-08-10    0.00002
2015-09-09    0.00002
2015-11-10    0.00002
2015-12-05    0.00002
2015-12-30    0.00002
2016-01-03    0.00002
2016-01-07    0.00002
2016-01-10    0.00004
2016-01-13    0.00002
2016-01-14    0.00002
2016-01-16    0.00002
2016-01-22    0.00002
2016-01-27    0.00006
2016-01-29    0.00002
2016-02-01    0.00002
2016-02-02    0.00004
2016-02-05    0.00004
2016-02-07    0.00002
2016-02-08    0.00002
2016-02-09    0.00004
2016-02-11    0.00002
2016-02-12    0.00006
2016-02-14    0.00004
2016-02-16    0.00002
2016-02-17    0.00002
2016-02-18    0.00004
2016-02-19    0.00006
2016-02-20    0.00004
2016-02-21    0.00006
               ...   
2016-03-09    0.03324
2016-03-10    0.03186
2016-03-11    0.03278
2016-03-12    0.03662
2016-03-13    0.01692
2016-03-14    0.03522
2016-03-15    0.03374
2016-03-16    0.03000
2016-03-17    0.03120
2016-03-18    0.01372
2016-03-19    0.03384
2016-03-20    0.03786
2016-03-21    0.03772
2016-03-22    0.03280
2016-03-23

There is a large variety of ad_created dates. Most occured within 1-2 months of the listing date, but a few are quite old, with the oldest at around 9 months.

In [32]:
autos['last_seen'].str[:10].value_counts(normalize=True, dropna=False).sort_index()

2016-03-05    0.00108
2016-03-06    0.00442
2016-03-07    0.00536
2016-03-08    0.00760
2016-03-09    0.00986
2016-03-10    0.01076
2016-03-11    0.01252
2016-03-12    0.02382
2016-03-13    0.00898
2016-03-14    0.01280
2016-03-15    0.01588
2016-03-16    0.01644
2016-03-17    0.02792
2016-03-18    0.00742
2016-03-19    0.01574
2016-03-20    0.02070
2016-03-21    0.02074
2016-03-22    0.02158
2016-03-23    0.01858
2016-03-24    0.01956
2016-03-25    0.01920
2016-03-26    0.01696
2016-03-27    0.01602
2016-03-28    0.02086
2016-03-29    0.02234
2016-03-30    0.02484
2016-03-31    0.02384
2016-04-01    0.02310
2016-04-02    0.02490
2016-04-03    0.02536
2016-04-04    0.02462
2016-04-05    0.12428
2016-04-06    0.22100
2016-04-07    0.13092
Name: last_seen, dtype: float64

The crawler recorded the date it last saw any listing, which allows us to determine on what day a listing was removed, precedently known as the car was sold.
The last three days contain a disproportionate amount of 'last seen' values. Given that these are 6-10x the values from the previous days, it's unlikely that there was a massive spike in sales, and more likely that these values are to do with the crawling period ending and don't indicate car sales.

In [15]:
autos['registration_year'].describe()

count    48565.000000
mean      2004.755421
std         88.643887
min       1000.000000
25%       1999.000000
50%       2004.000000
75%       2008.000000
max       9999.000000
Name: registration_year, dtype: float64

The year that the car was first registered will likely indicate the age of the car. Looking at this column, we note some odd values. The minimum value is 1000, long before cars were invented and the maximum is 9999, many years into the future. Because a car can't be first registered after the listing was seen, any vehicle with a registration year above 2016 is definitely inaccurate. Determining the earliest valid year is more difficult. Realistically, it could be somewhere in the first few decades of the 1900s.

One option is to remove the listings with these values. Let's determine what percentage of our data has invalid values in this column:

In [16]:
(~autos['registration_year'].between(1930,2016)).sum() / autos.shape[0]

0.038937506434675175

Since this is less than 4% of our data, we will remove these rows.

In [17]:
autos = autos[autos['registration_year'].between(1930,2016)]#simplified code
#autos = autos[(autos['registration_year'] >= 1930) & (autos['registration_year'] <= 2016)]
autos['registration_year'].value_counts(normalize=True,ascending=False).head(10)

2000    0.067618
2005    0.062904
1999    0.062069
2004    0.057912
2003    0.057827
2006    0.057205
2001    0.056477
2002    0.053263
1998    0.050628
2007    0.048785
Name: registration_year, dtype: float64

In [105]:
autos['registration_year'].describe()


count    48017.000000
mean      2002.824021
std          7.184215
min       1931.000000
25%       1999.000000
50%       2003.000000
75%       2008.000000
max       2016.000000
Name: registration_year, dtype: float64

It is certain that most of the vehicles were first registered in the past 20 years.

Explore 'Price' by brand

In [146]:
autos['brand'].value_counts(normalize=True, ascending = False)

volkswagen        0.21374
opel              0.10922
bmw               0.10858
mercedes_benz     0.09468
audi              0.08566
ford              0.06958
renault           0.04808
peugeot           0.02912
fiat              0.02616
seat              0.01882
skoda             0.01572
mazda             0.01514
nissan            0.01508
citroen           0.01402
smart             0.01402
toyota            0.01234
sonstige_autos    0.01092
hyundai           0.00976
volvo             0.00914
mini              0.00848
mitsubishi        0.00812
honda             0.00798
kia               0.00712
alfa_romeo        0.00658
porsche           0.00588
suzuki            0.00586
chevrolet         0.00566
chrysler          0.00362
dacia             0.00258
daihatsu          0.00256
jeep              0.00220
subaru            0.00218
land_rover        0.00198
saab              0.00160
daewoo            0.00158
trabant           0.00156
jaguar            0.00154
rover             0.00138
lancia      

In [19]:
autos['brand'].describe()

count          46674
unique            40
top       volkswagen
freq            9862
Name: brand, dtype: object

German manufacturers represent four out of the top five brands, almost 50% of the overall listings. Volkswagen is by far the most popular brand out of 40 brands, with approximately double the cars for sale of the next two brands combined.
There are lots of brands that don't have a significant percentage of listings, so we will limit our analysis to brands representing more than 5% of total listings.

In [20]:
count_brands = autos['brand'].value_counts(normalize=True)
popular_brands = count_brands[count_brands>.05].index
print(popular_brands)

Index(['volkswagen', 'bmw', 'opel', 'mercedes_benz', 'audi', 'ford'], dtype='object')


In [41]:
aggregate_brands_price={}
for brand in popular_brands:
    brand_name = autos[autos['brand']==brand]
    price_mean = brand_name['price'].mean()
    aggregate_brands_price[brand]=int(price_mean)
print(aggregate_brands_price)

{'opel': 2944, 'audi': 9212, 'ford': 3728, 'bmw': 8261, 'mercedes_benz': 8536, 'volkswagen': 5332}


Audi is the most expensive with price significantly high, followed by Mercedes and BMW. Ford and Opel can be classified as cheap price range and Volkswagen is somewhere in between. 

For the top 6 brands, let's use aggregation to understand the average mileage for those cars and if there's any visible link with mean price. While our natural instinct may be to display both aggregated series objects and visually compare them, this has a few limitations:

it's difficult to compare more than two aggregate series objects if we want to extend to more columns
we can't compare more than a few rows from each series object
we can only sort by the index (brand name) of both series objects so we can easily make visual comparisons

In [51]:
car_series=pd.Series(aggregate_brands_price)
print(car_series)


audi             9212
bmw              8261
ford             3728
mercedes_benz    8536
opel             2944
volkswagen       5332
dtype: int64


In [52]:
pd.DataFrame(car_series,columns=['price_mean'])

,price_mean
audi,9212
bmw,8261
ford,3728
mercedes_benz,8536
opel,2944
volkswagen,5332


In [46]:
brand_mean_mileage={}
for brand in popular_brands:
    brand_name = autos[autos['brand']==brand]
    mileage_mean = brand_name['odometer_km'].mean()
    brand_mean_mileage[brand]=int(mileage_mean)

In [47]:
mean_mileage=pd.Series(brand_mean_mileage).sort_values(ascending=False)
mean_price=pd.Series(aggregate_brands_price).sort_values(ascending=False)
print(mean_price)
print(mean_mileage)

audi             9212
mercedes_benz    8536
bmw              8261
volkswagen       5332
ford             3728
opel             2944
dtype: int64
bmw              132682
mercedes_benz    130796
audi             129492
opel             129383
volkswagen       128896
ford             124349
dtype: int64


In [48]:
brand_data=pd.DataFrame(mean_mileage, columns=['mileage_mean'])
brand_data

,mileage_mean
bmw,132682
mercedes_benz,130796
audi,129492
opel,129383
volkswagen,128896
ford,124349


In [50]:
brand_data["price_mean"] = mean_price
brand_data

,mileage_mean,price_mean
bmw,132682,8261
mercedes_benz,130796,8536
audi,129492,9212
opel,129383,2944
volkswagen,128896,5332
ford,124349,3728


The range of car mileages does not vary as much as the prices do by brand, instead all falling within 10% for the top brands. There is a slight trend to the more expensive vehicles having higher mileage, with the less expensive vehicles having lower mileage.